<a href="https://colab.research.google.com/github/Muyiiwaa/machine_learning_notes/blob/master/multilayer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### REGRESSION NEURAL NETWORK

In [5]:
import torch
import numpy as np
from torch import nn, optim

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

In [6]:
# simple differentiation with pytorch
x = torch.tensor(data = 4.0, dtype = torch.float32, requires_grad=True)
y = torch.tensor(data = 3.0, dtype = torch.float32, requires_grad=True)

f = 3 * x ** 2 + 5 * x + 6 + 2 * y

f.backward()

print(x.grad)
y.grad

tensor(29.)


tensor(2.)

In [12]:
# prepare the dataset

X = load_diabetes()['data']
y = load_diabetes()['target']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 23)

# convert all the arrays to tensors

X_train_tensor = torch.tensor(data = X_train, dtype = torch.float32)
X_test_tensor = torch.tensor(data = X_test, dtype = torch.float32)
y_train_tensor = torch.tensor(data = y_train, dtype = torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(data = y_test, dtype = torch.float32).view(-1, 1)

In [50]:
# define the neural net architecture

class Diabetes_NN(nn.Module):

  # define the constructor
  def __init__(self, input_dim: int) -> None:
    super().__init__()
    self.fc1 = nn.Linear(in_features=input_dim, out_features=900)
    self.fc2 = nn.Linear(in_features=900, out_features=500)
    self.fc3 = nn.Linear(in_features=500, out_features=250)
    self.fc4 = nn.Linear(in_features=250, out_features=50)
    self.out = nn.Linear(in_features=50, out_features= 1)
    self.relu = nn.ReLU()

  def forward(self, X:torch.Tensor):
    result = self.fc1(X)
    result = self.relu(result)
    result = self.relu(self.fc2(result))
    result = self.relu(self.fc3(result))
    result = self.relu(self.fc4(result))
    result = self.out(result)

    return result

In [51]:
# step 3: Initialize the model and the model hyper-parameters
model = Diabetes_NN(input_dim=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(params = model.parameters(), lr = 0.0001)
EPOCHS = 5000

In [52]:
# step 4: set up the training loop

model.train()
for epoch in range(EPOCHS):
  # forward pass
  train_preds = model(X_train_tensor)
  loss = criterion(train_preds, y_train_tensor)

  # back propagation
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 1000 == 0:
    print(f'epoch: {(epoch/1000) + 1}/{EPOCHS/1000}.....LOSS: {loss.item()}')

epoch: 1.0/5.0.....LOSS: 27716.220703125
epoch: 2.0/5.0.....LOSS: 3008.132080078125
epoch: 3.0/5.0.....LOSS: 2606.08642578125
epoch: 4.0/5.0.....LOSS: 2309.51318359375
epoch: 5.0/5.0.....LOSS: 1671.867431640625


In [57]:
loss.item() ** 0.5

27.153539313791583

In [54]:
# step 5. get evaluation

model.eval()
with torch.no_grad():
  test_preds = model(X_test_tensor)
  test_preds = test_preds.squeeze(1)
  test_preds = test_preds.detach().numpy()

In [55]:
print(root_mean_squared_error(y_test, test_preds))

67.23359315984835


###  CLASSIFICATION

In [92]:
import torch
import numpy as np
from torch import nn, optim

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,classification_report

In [58]:
# prepare the dataset

X = load_iris()['data']
y = load_iris()['target']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 23,
                                                    stratify=y)

# convert all the arrays to tensors

X_train_tensor = torch.tensor(data = X_train, dtype = torch.float32)
X_test_tensor = torch.tensor(data = X_test, dtype = torch.float32)
y_train_tensor = torch.tensor(data = y_train, dtype = torch.long)
y_test_tensor = torch.tensor(data = y_test, dtype = torch.long)

In [62]:
# define the neural net architecture

class Flower_NN(nn.Module):

  # define the constructor
  def __init__(self, input_dim: int) -> None:
    super().__init__()
    self.fc1 = nn.Linear(in_features=input_dim, out_features=600)
    self.fc2 = nn.Linear(in_features=600, out_features=300)
    self.fc3 = nn.Linear(in_features=300, out_features=150)
    self.fc4 = nn.Linear(in_features=150, out_features=50)
    self.out = nn.Linear(in_features=50, out_features= 3)
    self.relu = nn.ReLU()

  def forward(self, X:torch.Tensor):
    result = self.fc1(X)
    result = self.relu(result)
    result = self.relu(self.fc2(result))
    result = self.relu(self.fc3(result))
    result = self.relu(self.fc4(result))
    result = self.out(result)

    return result

In [63]:
# step 3: Initialize the model and the model hyper-parameters
model = Flower_NN(input_dim=X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(), lr = 0.0001)
EPOCHS = 5000

In [64]:
# step 4: set up the training loop

model.train()
for epoch in range(EPOCHS):
  # forward pass
  train_preds = model(X_train_tensor)
  loss = criterion(train_preds, y_train_tensor)

  # back propagation
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 1000 == 0:
    print(f'epoch: {(epoch/1000) + 1}/{EPOCHS/1000}.....LOSS: {loss.item()}')

epoch: 1.0/5.0.....LOSS: 1.100999116897583
epoch: 2.0/5.0.....LOSS: 0.03135601431131363
epoch: 3.0/5.0.....LOSS: 0.006266219075769186
epoch: 4.0/5.0.....LOSS: 0.0006185732781887054
epoch: 5.0/5.0.....LOSS: 0.0001516809716122225


In [94]:
# step 5. get evaluation

model.eval()
with torch.no_grad():
  test_preds = model(X_test_tensor)
  test_logits, test_preds = torch.max(test_preds, 1)
  test_preds = test_preds.detach().numpy()



print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [65]:
y_test_tensor

tensor([0, 0, 0, 1, 2, 1, 2, 1, 0, 2, 1, 2, 0, 2, 2, 1, 2, 1, 1, 2, 0, 2, 2, 0,
        0, 0, 0, 1, 1, 1])

In [75]:
test_preds[:5]

tensor([[ 49.3684,  34.1302, -45.4770],
        [ 47.0161,  32.8743, -43.5847],
        [ 50.5941,  34.8095, -46.4815],
        [  1.9646,  21.5952, -16.3325],
        [-10.4851, -10.1355,  17.5664]])

In [73]:
torch.max(test_preds, 1)

torch.return_types.max(
values=tensor([49.3684, 47.0161, 50.5941, 21.5952, 17.5664, 18.0786, 13.4516, 21.1595,
        46.0909, 17.5906, 12.1758,  8.6851, 50.9490, 10.2815,  5.5728, 22.1788,
         5.4056, 21.7586, 14.0211, 15.1917, 50.9421,  6.4043, 18.4930, 45.8903,
        47.1235, 50.7822, 44.6392, 21.6050, 20.6353,  7.1746]),
indices=tensor([0, 0, 0, 1, 2, 1, 2, 1, 0, 2, 1, 2, 0, 2, 2, 1, 2, 1, 1, 2, 0, 2, 2, 0,
        0, 0, 0, 1, 1, 1]))

In [101]:
model.eval()
with torch.no_grad():
  test_preds = model(X_test_tensor)

test_preds[:5]

tensor([[ 49.3684,  34.1302, -45.4770],
        [ 47.0161,  32.8743, -43.5847],
        [ 50.5941,  34.8095, -46.4815],
        [  1.9646,  21.5952, -16.3325],
        [-10.4851, -10.1355,  17.5664]])

In [76]:
softmax = nn.Softmax()

In [102]:
softmax(test_preds)[:5]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[1.0000e+00, 2.4106e-07, 6.4446e-42],
        [1.0000e+00, 7.2157e-07, 4.4932e-40],
        [1.0000e+00, 1.3958e-07, 6.9224e-43],
        [2.9823e-09, 1.0000e+00, 3.3744e-17],
        [6.5673e-13, 9.3157e-13, 1.0000e+00]])

In [99]:
test_logits

tensor([49.3684, 47.0161, 50.5941, 21.5952, 17.5664, 18.0786, 13.4516, 21.1595,
        46.0909, 17.5906, 12.1758,  8.6851, 50.9490, 10.2815,  5.5728, 22.1788,
         5.4056, 21.7586, 14.0211, 15.1917, 50.9421,  6.4043, 18.4930, 45.8903,
        47.1235, 50.7822, 44.6392, 21.6050, 20.6353,  7.1746])